# 文書分類

前回までの自然言語処理の基礎を活かして、実際の文書分類を行うためのデータの準備を行いましょう。

## 問題設定

今回は、ニュース記事をカテゴリ分けする問題設定で考えていきます。
まずはじめに、特定フォルダ内の記事を読み込む練習をしましょう。

![](images/4-5/img.png)

今回はこのように９つの記事カテゴリーを文章から分類していきます。

## データの前準備

まず、zipファイルを解凍する必要があります。
Jupyter Notebookでは、zipファイルなどで圧縮してからUploadする必要があるため、圧縮してUploadして解凍するといった操作を多用しますので、覚えておきましょう。

`!` を使用することで、LinuxのBashコマンドを使用することができ、こちらで`unzip`コマンドを使うことで簡単にファイル操作が可能です。一番楽です。

まずはファイルが格納されているか確認しましょう。

In [1]:
!ls

Subclassing.ipynb                   _DS_Store
SubclassingAPI.ipynb                azure.ipynb
TF_Classification.ipynb             data
TF_Regression.ipynb                 image.ipynb
TF_Seq2Seq.ipynb                    images
TF_TensorBoard.ipynb                intro.ipynb
TF_cat_dog.ipynb                    livedoordic.txt
TF_cnn_cifar10.ipynb                peachy
TF_cnn_mnist.ipynb                  programming.ipynb
TF_document_classification.ipynb    registration.ipynb
TF_nlp.ipynb                        send_img.py
TF_time_series.ipynb                text
TF_time_series_classification.ipynb topic-news


In [ ]:
!unzip text.zip

In [2]:
# ファイルを確認
!ls -a text

.              it-life-hack   livedoor-homme peachy         sports-watch
..             kaden-channel  movie-enter    smax           topic-news


`.DS_Store`という不必要なファイルが存在するため、消しておきましょう。
圧縮のタイミングやOSによるものでいらないファイルが見つかるかもしれないため、フォルダの中などのデータをよく見ておきましょう。

In [3]:
!rm text/.DS_Store

rm: text/.DS_Store: No such file or directory


In [4]:
# ファイルを確認
!ls -a text

.              it-life-hack   livedoor-homme peachy         sports-watch
..             kaden-channel  movie-enter    smax           topic-news


このように削除できました。

## 各ディレクトリのテキスト情報を変数に読み込む

このテキスト情報を読み込むときに、便利なものが、辞書型の変数になります。

辞書型（data）のキーにファイル名、値にテキストの文字列を読み込みます。

ファイルやディレクトリを操作するときにとても便利な`glob`というライブラリもインポートしておきましょう。

In [1]:
from glob import glob

In [2]:
import pandas as pd

In [3]:
directories = glob('text/*')
directories

['text/smax',
 'text/kaden-channel',
 'text/livedoor-homme',
 'text/movie-enter',
 'text/sports-watch',
 'text/it-life-hack',
 'text/topic-news',
 'text/peachy']

for文で繰り返すときに、各フォルダ（ディレクトリ）に対応する要素番号を分類に使用するラベルと設定したいため、`enumerate`を使うことで、要素番号も併せて取得しておきましょう。

In [4]:
for (i, directory) in enumerate(directories):
    print(i)
    print(directory)
    print('- - -')

0
text/smax
- - -
1
text/kaden-channel
- - -
2
text/livedoor-homme
- - -
3
text/movie-enter
- - -
4
text/sports-watch
- - -
5
text/it-life-hack
- - -
6
text/topic-news
- - -
7
text/peachy
- - -


In [5]:
texts, labels = [], []
for (i, directory) in enumerate(directories):
    #各ディレクトリ内のtxtファイルのパスをすべて取得
    filepaths = glob('{}/*.txt'.format(directory))
    # テキストを読み込んで、内容をtextに格納、ラベルも併せて格納
    for filepath in filepaths:
        with open(filepath, encoding='utf-8') as f:
            text = ''.join(f.readlines()[2:])  # URL等の先頭２行を除いた各行の文章を連結（join）して格納
            texts.append(text)
            labels.append(i)

たとえば、最後に格納された文書を確認してみましょう。

In [6]:
texts[0]

'東京スカイツリーを好きな色でライトアップしちゃおう！「夜のスカイツリー 〜 ライトアップ時計デザイナー 無料」【iPhoneアプリ】\n何色のライトアップが綺麗かな！？ \n\n賑わいをみせている東京スカイツリーですが、夜の顔ともいえるライトアップも綺麗ですよね。水色をベースとした「粋」、紫色をベースとした「雅」も印象がガラッと変わっていい感じになりますよね！\n\nこの色を使ってライトアップをしたらもっと綺麗になると思うんだけど…なんて思った方にオススメのアプリが今回紹介するiPhone向けアプリ「夜のスカイツリー 〜 ライトアップ時計デザイナー 無料」です！\n\nこのアプリを使うと好きな色で東京スカイツリーをライトアップさせることができます♪\n\n日付や時計も表示できる時計アプリとしても使うことができ、綺麗にカッコよくライトアップできたら壁紙に設定という使い方もできちゃいます。\n\nでは、早速、紹介してみたいと思います。\n\n\nアプリを起動してしばし待ちます…。\n\n\n待っていると徐々に明かりが灯ってきます。\n\n\nそしてスカイツリーが現れます！\nちなみにこのイルミネーションは「雅」紫色のデコレーションが綺麗ですね。\n\n\nそしてこちらが「粋」ブルーのイルミネーションになっています。\n\n好きな色のイルミネーションを設定してみましょう！\n画面をタップするとイルミネーションの色を細かく好きな色に設定することができます。\n日付や時計も個々に色を変えることができ、右上の雅、粋をタップするとそれぞれの初期設定イルミネーションに上書き保存されます。\n個々に色を変えるときは、左右の縦に並んだアイコンをタップして左下にあるカラーダイヤルで色を決め、右のダイヤルでグラデーションの濃さを決めていき右下の▶をタップしたら設定完了です。\n\n\n\n\n\n\n\n設定した色でスカイツリーがライトアップされました！\n\n\n\n\n日付や時計、スカイツリーをタップして好きな場所に移動させることができます。\n\n色編集画面右上にある歯車にようなアイコンをタップすると時計や日付の大きさ、秒表示、日付の書式などの細かい設定をすることができるのでいろいろと設定を変えてみるのも楽しいですよ♪\n\n\n\n\n\n色編集画面の背景をスワイプすると用意され

In [7]:
labels[0]

0

## 文章から名詞のみを抽出

前に作成した名詞抽出用の関数を使用して、文書全体で使用されている名詞を全てword_collectというリストに格納していきましょう。

In [8]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libboost-filesystem1.65.1 libboost-iostreams1.65.1
  libboost-system1.65.1 libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblocale-gettext-perl liblwp-mediatypes-perl
  libparse-debianchangelog-perl libsigc++-2.0-0v5 libsub-name-perl
  libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libboost-filesystem1.65.1 libboost-iostreams1.65.1
  libboost-system1.65.1 libcgi-fast-perl libcgi-pm-perl libclass-a

In [9]:
import MeCab

In [10]:
mecab = MeCab.Tagger('-Ochasen')

In [11]:
def get_nouns(text):
    nouns = []
    res = mecab.parse(text)
    words = res.split('\n')[:-2] #EOSと空白部分の削除
    for word in words:
        part = word.split('\t')
        if '名詞' in part[3]:
            nouns.append(part[0])
    return nouns

In [12]:
word_collect = []
for text in texts:
    nouns = get_nouns(text)
    word_collect.append(nouns)
    
# ワンライナーで下記のように書いてもOK
# word_collect = [ get_nouns(text) for text in texts ]

## BoWに変換

前回と同様に`gensim`を使いましょう。

In [13]:
from gensim import corpora, matutils

まずはBoW用の辞書を作りましょう。

In [14]:
dictionary = corpora.Dictionary(word_collect)

In [15]:
len(dictionary)

81051

今回は`58205`単語あるようです。

このままでも良いのですが、出現回数が多すぎたり引きすぎる単語をフィルタリングすると、特徴のある単語のみに絞ることができます。

In [16]:
dictionary.filter_extremes(no_below=20)

In [17]:
len(dictionary)

6697

このように全体で20回以上出現しない単語はフィルタリングすることで、`6586`単語に抑えることができました。
辞書作成に多少時間がかかるため、あとから使えるようにこの段階で保存しておくと良いでしょう。

In [18]:
# 後から使えるように保存しておく
dictionary.save_as_text('livedoordic.txt')

それではこの作成した辞書を使って、BoWに変換しましょう。

In [19]:
n_words = len(dictionary)

In [20]:
# BOWによる特徴ベクトルの作成
x = []
for nouns in word_collect:
    bow_id = dictionary.doc2bow(nouns)
    bow = matutils.corpus2dense([bow_id], n_words).T[0]
    x.append(bow)

In [21]:
import numpy as np

In [22]:
x = np.array(x)
t = np.array(labels)

In [23]:
x.shape

(6505, 6697)

In [24]:
t.shape

(6505,)

こちらのように機械学習で使用できる形式へ変換できました。
あとは、単純な分類問題であるため、NNでクラス分類することで実装できます。

## 演習課題

得られたデータセットを使用してクラス分類を行うNNのモデルを作成せよ。

条件

- seedは0で固定
- 全体の70%が訓練データ、残りの30%が検証データ（ランダムに分割）

In [25]:
import os
import random

def reset_seed(seed=0):
    
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed) #　random関数のシードを固定
    np.random.seed(seed) #numpyのシードを固定
    tf.random.set_seed(seed) #tensorflowのシードを固定

In [26]:
# Pythonの挙動を整えるライブラリのインポート
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow と tf. のインポート
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import models, layers

In [27]:
print(tf.__version__)

2.0.0


In [28]:
from sklearn.model_selection import train_test_split

# 訓練データと検証データの分割
train_x, val_x, train_t, val_t = train_test_split(x, t, test_size=0.3, random_state=0)

In [29]:
# シードの固定
reset_seed(0)

model = models.Sequential()

model.add(layers.Dense(200, input_shape=(6586, ), activation='relu'))
model.add(layers.Dense(9, activation='softmax'))

optimizer = keras.optimizers.SGD(lr=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [30]:
history = model.fit(train_x, train_t,
          batch_size=100,
          epochs=50,
          verbose=1,
          validation_data=(val_x, val_t))

ValueError: Error when checking input: expected dense_input to have shape (6586,) but got array with shape (6697,)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 学習結果をPandasのDataFrame型で読み込みます。
results = pd.DataFrame(history.history)

# accuracy（精度）を表示
results[['accuracy', 'val_accuracy']].plot()

In [ ]:
# loss（損失関数）を表示
results[['loss', 'val_loss']].plot()